# Load Data

In [1]:
# Base / Native
import math
import os
from os.path import join
import pickle
import re
import warnings
warnings.filterwarnings('ignore')

# Numerical / Array
import lifelines
from lifelines.statistics import logrank_test
from lifelines.utils import concordance_index as ci
from sksurv.metrics import cumulative_dynamic_auc
import numpy as np
import pandas as pd
import scipy
from scipy import interp
from scipy.stats import ttest_ind
from tqdm import tqdm

### Table 2

In [79]:
results_all = {}
tcga_studies = ['tcga_brca', 'tcga_coadread', 'tcga_kirc', 'tcga_kirp', 'tcga_luad', 'tcga_stad']

dataroot = './results_surv/5foldcv/'
for exp_dir in sorted(os.listdir(dataroot)):
    results = []
    for study in tcga_studies:
        study_dir = os.path.join(dataroot, exp_dir, '%s_%s_s1' % (study, exp_dir))
        if os.path.isdir(study_dir) and 'summary_latest.csv' in os.listdir(study_dir):
            cin = '%0.3f' % pd.read_csv(os.path.join(study_dir, 'summary_latest.csv'))['val_cindex'].mean()
            cin += ' +/- %0.3f' % pd.read_csv(os.path.join(study_dir, 'summary_latest.csv'))['val_cindex'].std()
            results.append(cin)
        else:
            results.append(np.NaN)
            
    results_all[exp_dir] = results

In [80]:
models = ['AMIL_nll_surv_a0.0_5foldcv_gc32_dino_sig',
          'MIFCN_nll_surv_a0.0_5foldcv_gc32_dino_sig',
          'DSMIL_nll_surv_a0.0_5foldcv_gc32_dino_sig',
          'DGCN_nll_surv_a0.0_5foldcv_gc32_dino_sig', 
          'HIPT_LGP[vit4k_xs_dino]_freeze_[None]_nll_surv_a0.0_5foldcv_gc32_dino_sig'
]

model_names = ['AMIL', 'MI-FCN', 'DGCN', 'DSMIL', 'HIPT']
results_df = pd.DataFrame(results_all).T
results_df.columns = tcga_studies
results_df = results_df.loc[models]
results_df.index = model_names
print(results_df.to_latex())
results_df

\begin{tabular}{lllllll}
\toprule
{} &        tcga\_brca &    tcga\_coadread &        tcga\_kirc &        tcga\_kirp &        tcga\_luad &        tcga\_stad \\
\midrule
AMIL   &  0.487 +/- 0.079 &  0.566 +/- 0.075 &  0.561 +/- 0.074 &  0.671 +/- 0.076 &  0.584 +/- 0.054 &  0.562 +/- 0.049 \\
MI-FCN &  0.472 +/- 0.023 &  0.561 +/- 0.088 &  0.521 +/- 0.084 &  0.472 +/- 0.162 &  0.563 +/- 0.037 &  0.563 +/- 0.067 \\
DGCN   &  0.534 +/- 0.060 &  0.538 +/- 0.049 &  0.591 +/- 0.093 &  0.636 +/- 0.066 &  0.592 +/- 0.070 &  0.513 +/- 0.069 \\
DSMIL  &  0.472 +/- 0.020 &  0.470 +/- 0.053 &  0.548 +/- 0.057 &  0.654 +/- 0.134 &  0.537 +/- 0.061 &  0.546 +/- 0.047 \\
HIPT   &  0.634 +/- 0.050 &  0.608 +/- 0.088 &  0.642 +/- 0.028 &  0.670 +/- 0.065 &  0.538 +/- 0.044 &  0.570 +/- 0.081 \\
\bottomrule
\end{tabular}



,tcga_brca,tcga_coadread,tcga_kirc,tcga_kirp,tcga_luad,tcga_stad
AMIL,0.487 +/- 0.079,0.566 +/- 0.075,0.561 +/- 0.074,0.671 +/- 0.076,0.584 +/- 0.054,0.562 +/- 0.049
MI-FCN,0.472 +/- 0.023,0.561 +/- 0.088,0.521 +/- 0.084,0.472 +/- 0.162,0.563 +/- 0.037,0.563 +/- 0.067
DGCN,0.534 +/- 0.060,0.538 +/- 0.049,0.591 +/- 0.093,0.636 +/- 0.066,0.592 +/- 0.070,0.513 +/- 0.069
DSMIL,0.472 +/- 0.020,0.470 +/- 0.053,0.548 +/- 0.057,0.654 +/- 0.134,0.537 +/- 0.061,0.546 +/- 0.047
HIPT,0.634 +/- 0.050,0.608 +/- 0.088,0.642 +/- 0.028,0.670 +/- 0.065,0.538 +/- 0.044,0.570 +/- 0.081
